In [1]:
import psycopg2
import pandas as pd

# Create a connection
conn = psycopg2.connect(
    host='db',  # Use the service name from Docker Compose as the hostname
    port=5432,
    dbname='tradedb',
    user='postgres',
    password='postgres'
)

# Create a cursor object
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM raw.trades")

# Fetch the results
rows = cur.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cur.description]

# Create a DataFrame
df = pd.DataFrame(rows, columns=columns)
df2 = pd.DataFrame(rows, columns=columns)

# Create alias
raw_trades_df = df

# Adding a date column to the DataFrame
raw_trades_df['date'] = raw_trades_df['timestamp'].dt.date

# Create a new column 'prev_side' to store the previous side value for each account_id, symbol, and date
raw_trades_df['prev_side'] = raw_trades_df.sort_values(by='timestamp').groupby(['account_id', 'symbol', 'date'])['side'].shift()

# Filter rows where side is 'sell' and prev_side is 'buy'
buy_sell_pairs_df = raw_trades_df[(raw_trades_df['side'] == 'sell') & (raw_trades_df['prev_side'] == 'buy')]

# Group by account_id and count the number of pattern day trades
pdt_count_df = buy_sell_pairs_df.groupby('account_id').size().reset_index(name='pdt_count')

# Sort by pdt_count in descending order
pdt_count_df.sort_values(by='pdt_count', ascending=False, inplace=True)

pdt_count_df

# Write DataFrame to CSV
pdt_count_df.to_csv('pdt_count.csv', index=False)

# Close the cursor and connection
cur.close()
conn.close()